# Central Hidroeléctrica Yacyretá
## 09I - Modelo de predicción con algoritmos de aprendizaje automático - Opcion 3
### Univariable, Feature-Engineering. Loopeado para varios fh y lags, que se configuran, asi como los regresores a usar.
#### Fecha: 12-Enero-22

In [1]:
# Para que funcione más rápidamente el intellisense
#%config Completer.use_jedi = False

In [2]:
#Cargo numpy y pandas para trabajar con dataframes
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 6)

In [3]:
# Imports necesarios para que funcione iplot() como método de pandas
%matplotlib inline
import plotly.graph_objs as go
from  plotly.offline import plot
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected='true')

In [4]:
# Planto una semilla para repetitibilidad
import random
random.seed(170162)
np.random.seed(170162)

In [5]:
import pprint
from tabulate import tabulate

In [6]:
from sktime.utils.plotting import plot_series
from sklearn.base import clone

In [7]:
# Importo los regresores
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso, Ridge  

In [8]:
# Métricas de evaluación de desempeño
from sktime.performance_metrics.forecasting import MeanAbsoluteError, MeanSquaredError, MeanAbsoluteScaledError

### Carpeta para los archivos '.csv' y gráficos

#### Nombre del Experimento

In [9]:
experimento = 'TFI 09I - Univariate Feature Engineering'

In [10]:
ahora = str(datetime.now()).replace(':', '-').replace(' ', '_')[:16]
path_grabacion = './output/' + experimento + '/'+ ahora + '/'
# Verifico que la carpeta exista y si no, la creo
import os
if not os.path.exists(path_grabacion):
    os.makedirs(path_grabacion)
    print('\nPath/carpeta de gráficos y de archivos creada:', path_grabacion)


Path/carpeta de gráficos y de archivos creada: ./output/TFI 09I - Univariate Feature Engineering/2022-06-15_20-56/


### Lectura del Dataset de la central hidroeléctrica Yacyretá

In [11]:
# Especifico el nombre del archivo con el dataset unificado de la CHY y el path
path = './datasets/CHY/output/'
file_name = 'CHY_dataset.csv'

In [12]:
# Leo el dataset creado en el notebook anterior
df = pd.read_csv(path + file_name)

In [13]:
# veo cabeza y cola del df
df

,FECHA,COTA_EMBALSE,COTA_RESTITUCION,SALTO,QAFLR,QVBP,QVBAC,QTURB,SINP,SADI
0,2011-01-01,82.04,60.17,21.87,13936.0,0.0,1045.0,11977.0,5391.00,49384.000
1,2011-01-02,82.06,60.17,21.89,13267.0,0.0,1045.0,11863.0,5631.60,48785.000
2,2011-01-03,82.07,60.12,21.95,12670.0,0.0,1045.0,11439.0,6871.60,46573.000
3,2011-01-04,82.08,60.12,21.96,12909.0,0.0,1045.0,11505.0,7460.80,46046.000
4,2011-01-05,82.13,60.08,22.05,13787.0,0.0,1047.0,11455.0,6555.00,46721.000
...,...,...,...,...,...,...,...,...,...,...
3828,2021-06-26,82.78,58.94,23.84,6545.0,0.0,1017.0,5157.0,4388.60,21594.112
3829,2021-06-27,82.78,58.94,23.84,6337.0,0.0,1017.0,5135.0,4188.42,21658.536
3830,2021-06-28,82.79,58.93,23.86,6138.0,0.0,1017.0,5121.0,5165.92,20606.040
3831,2021-06-29,82.79,58.93,23.86,6133.0,0.0,1017.0,5116.0,5427.90,20416.960


In [14]:
# Convierto la columna FECHA de object a datetime64
df['FECHA'] = pd.to_datetime(df['FECHA'])

In [15]:
# Hago que la columna FECHA sea el índice
df.set_index('FECHA', inplace=True)

In [16]:
# Si fuera necesario imputar NAN
df = df.asfreq('D', method='ffill')

### Feature Engineering

##### Genero los lags (hasta 99)

In [17]:
lags = 40
for i in range(1, lags+1):
    df['SALTO-' + str(i).zfill(2)] = df['SALTO'].shift(periods=i)

In [18]:
### Suma

# Suma rolling a 3 días
df['sum3'] = df['SALTO-01'].rolling(window=3).sum()

# Suma rolling a 5 días
df['sum5'] = df['SALTO-01'].rolling(window=5).sum()

# Suma rolling a 7 días
df['sum7'] = df['SALTO-01'].rolling(window=7).sum()

### Media

# Media de 3 días
df['media3'] = df['SALTO-01'].rolling(window=3).mean()

# Media de 5 días
df['media5'] = df['SALTO-01'].rolling(window=5).mean()

# Media de 7 días
df['media7'] = df['SALTO-01'].rolling(window=7).mean()

### Desviación std.

# Desviación estándard a 3 días
df['std3'] = df['SALTO-01'].rolling(window=3).std()

# Desviación estándard a 5 días
df['std5'] = df['SALTO-01'].rolling(window=5).std()

# Desviación estándard a 7 días
df['std7'] = df['SALTO-01'].rolling(window=7).std()

### Varianza

# Varianza a 3 días
df['var3'] = df['SALTO-01'].rolling(window=3).var()

# Varianza a 5 días
df['var5'] = df['SALTO-01'].rolling(window=5).var()

# Varianza a 7 días
df['var7'] = df['SALTO-01'].rolling(window=7).var()

### Correlación

# Correlación de 3 días
df['corr3'] = df['SALTO-01'].rolling(window=3).corr()

# Correlación de 5 días
df['corr5'] = df['SALTO-01'].rolling(window=5).corr()

# Correlación de 7 días
df['corr7'] = df['SALTO-01'].rolling(window=7).corr()

### Mínimo

# Minimo rolling a 3 días
df['min3'] = df['SALTO-01'].rolling(window=3).min()

# Minimo rolling a 5 días
df['min5'] = df['SALTO-01'].rolling(window=5).min()

# Minimo rolling a 7 días
df['min7'] = df['SALTO-01'].rolling(window=7).min()

### Máximo

# Máximo rolling a 3 días
df['max3'] = df['SALTO-01'].rolling(window=3).max()

# Máximo rolling a 5 días
df['max5'] = df['SALTO-01'].rolling(window=5).max()

# Máximo rolling a 7 días
df['max7'] = df['SALTO-01'].rolling(window=7).max()

### Error estandard de la media 

# Rolling standard error of mean 3 días
df['sem3'] = df['SALTO-01'].rolling(window=3, min_periods=0).sem()

# Rolling starndard error of mean 5 días
df['sem5'] = df['SALTO-01'].rolling(window=5, min_periods=0).sem()

# Rolling starndard error of mean 7 días
df['sem7'] = df['SALTO-01'].rolling(window=7, min_periods=0).sem()

### Covarianza

# Rolling covariance a 3 días
df['cov3'] = df['SALTO-01'].rolling(window=3).cov()

# Rolling covariance a 5 días
df['cov5'] = df['SALTO-01'].rolling(window=5).cov()

# Rolling covariance a 7 días
df['cov7'] = df['SALTO-01'].rolling(window=7).cov()

### Mediana

# Rolling mediana a 3 días
df['med3'] = df['SALTO-01'].rolling(window=3).median()

# Rolling mediana a 5 días
df['med5'] = df['SALTO-01'].rolling(window=5).median()

# Rolling mediana a 7 días
df['med7'] = df['SALTO-01'].rolling(window=7).median()

### Expanding

# Expanding media a 3 días
df['emean3'] = df['SALTO-01'].expanding(min_periods=3).mean()

# Expanding media a 5 días
df['emean5'] = df['SALTO-01'].expanding(min_periods=5).mean()

# Expanding media a 5 días
df['emean7'] = df['SALTO-01'].expanding(min_periods=7).mean()

### Features relacionadas a la fecha

# Agrego el año, el nro. de semana y el día de la semana
df[['año', 'semana', 'dia_semana']] = np.array(df.index.isocalendar().values, dtype=float)
df[['año', 'semana', 'dia_semana']] = df[['año', 'semana', 'dia_semana']].shift(periods=1)

# Agrego el mes
df['quarter'] = df.index.quarter
df['quarter'] = df['quarter'].shift(periods=1)

# Agrego el mes del año
df['mes'] = df.index.month
df['mes'] = df['mes'].shift(periods=1)

### Posicionales

# Agrego el índice en el dataframe
df['posicion'] = [i for i in range(0, len(df))]

### Diferencias

#df['SALTO-01delta-02'] = df['SALTO-01'] - df['SALTO-02']

### Derivadas

# Derivada primera
#df['delta08'] = df['SALTO-08'] - df['SALTO-09']

# Derivada segunda
#df['delta02_01'] = (df['SALTO-01'] - df['SALTO-02']) - (df['SALTO-02'] - df['SALTO-03'])

# derivada 1ra
df['deriv1SALTO-01'] = df['SALTO-01'].diff()

# derivada 2da
df['deriv2SALTO-01'] = df['SALTO-01'].diff().diff()

# derivada 3ra
df['deriv3SALTO-01'] = df['SALTO-01'].diff().diff().diff()

# derivada 4ta
df['deriv4SALTO-01'] = df['SALTO-01'].diff().diff().diff().diff()

# derivada 5ta
df['deriv5SALTO-01'] = df['SALTO-01'].diff().diff().diff().diff().diff()

### Gradiente

#df['pendienteSALTO-01'] = pd.Series(np.gradient(df['SALTO-01']), df['SALTO-01'].index, name='slope')
# df['pendienteSALTO-02'] = pd.Series(np.gradient(df['SALTO-02']), df['SALTO-02'].index, name='slope')
# df['pendienteSALTO-03'] = pd.Series(np.gradient(df['SALTO-03']), df['SALTO-03'].index, name='slope')
# df['pendienteSALTO-04'] = pd.Series(np.gradient(df['SALTO-04']), df['SALTO-04'].index, name='slope')
# df['pendienteSALTO-05'] = pd.Series(np.gradient(df['SALTO-05']), df['SALTO-05'].index, name='slope')
# df['pendienteSALTO-06'] = pd.Series(np.gradient(df['SALTO-06']), df['SALTO-06'].index, name='slope')

In [19]:
df.columns

Index(['COTA_EMBALSE', 'COTA_RESTITUCION', 'SALTO', 'QAFLR', 'QVBP', 'QVBAC',
       'QTURB', 'SINP', 'SADI', 'SALTO-01', 'SALTO-02', 'SALTO-03', 'SALTO-04',
       'SALTO-05', 'SALTO-06', 'SALTO-07', 'SALTO-08', 'SALTO-09', 'SALTO-10',
       'SALTO-11', 'SALTO-12', 'SALTO-13', 'SALTO-14', 'SALTO-15', 'SALTO-16',
       'SALTO-17', 'SALTO-18', 'SALTO-19', 'SALTO-20', 'SALTO-21', 'SALTO-22',
       'SALTO-23', 'SALTO-24', 'SALTO-25', 'SALTO-26', 'SALTO-27', 'SALTO-28',
       'SALTO-29', 'SALTO-30', 'SALTO-31', 'SALTO-32', 'SALTO-33', 'SALTO-34',
       'SALTO-35', 'SALTO-36', 'SALTO-37', 'SALTO-38', 'SALTO-39', 'SALTO-40',
       'sum3', 'sum5', 'sum7', 'media3', 'media5', 'media7', 'std3', 'std5',
       'std7', 'var3', 'var5', 'var7', 'corr3', 'corr5', 'corr7', 'min3',
       'min5', 'min7', 'max3', 'max5', 'max7', 'sem3', 'sem5', 'sem7', 'cov3',
       'cov5', 'cov7', 'med3', 'med5', 'med7', 'emean3', 'emean5', 'emean7',
       'año', 'semana', 'dia_semana', 'quarter', 'mes', 'posi

In [20]:
features_generadas = ['sum3', 'sum5', 'sum7', 'media3', 'media5', 'media7', 'std3', 'std5',
       'std7', 'var3', 'var5', 'var7', 'corr3', 'corr5', 'corr7', 'min3',
       'min5', 'min7', 'max3', 'max5', 'max7', 'sem3', 'sem5', 'sem7', 'cov3',
       'cov5', 'cov7', 'med3', 'med5', 'med7', 'emean3', 'emean5', 'emean7',
       'año', 'semana', 'dia_semana', 'quarter', 'mes', 'posicion',
       'deriv1SALTO-01', 'deriv2SALTO-01', 'deriv3SALTO-01', 'deriv4SALTO-01',
       'deriv5SALTO-01']

In [21]:
features_exogenas = ['COTA_EMBALSE', 'COTA_RESTITUCION', 'QAFLR', 'QVBP', 'QVBAC',
       'QTURB', 'SINP', 'SADI']

####  Borro los registros que quedaron con un NAN al desplazarse

In [22]:
df.dropna(how='any', inplace=True)

### Límites del dataset

In [23]:
print('\n*** El dataset queda desde: ', df.index[0].strftime('%Y-%m-%d'), ', hasta: ', df.index[-1].strftime('%Y-%m-%d'))  


*** El dataset queda desde:  2011-02-10 , hasta:  2021-06-30


### Conjunto de regresores a utilizar

In [24]:
# Lista con los regresores
regresores = [ 
    KNeighborsRegressor()
    ,Lasso()
    ,LGBMRegressor(objective='regression', linear_tree=True)
    ,LinearRegression()
    ,MLPRegressor(max_iter=600, random_state=170162)
    ,RandomForestRegressor(criterion='absolute_error', random_state=170162) 
    ,Ridge()
    ,SVR(gamma='auto', kernel='rbf') 
    ,XGBRegressor()
    ]

### Métricas de evaluación de desempeño

In [25]:
mae = MeanAbsoluteError()
mse = MeanSquaredError()
rmse = MeanSquaredError(square_root=True)
mase = MeanAbsoluteScaledError()

### Conjuntos de Predictoras y de Target

#### Matriz de predictoras "X" 

In [26]:
# Me quedo con las columnas del df menos 'SALTO'
X = df.drop(columns=['SALTO'])

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3794 entries, 2011-02-10 to 2021-06-30
Freq: D
Data columns (total 92 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   COTA_EMBALSE      3794 non-null   float64
 1   COTA_RESTITUCION  3794 non-null   float64
 2   QAFLR             3794 non-null   float64
 3   QVBP              3794 non-null   float64
 4   QVBAC             3794 non-null   float64
 5   QTURB             3794 non-null   float64
 6   SINP              3794 non-null   float64
 7   SADI              3794 non-null   float64
 8   SALTO-01          3794 non-null   float64
 9   SALTO-02          3794 non-null   float64
 10  SALTO-03          3794 non-null   float64
 11  SALTO-04          3794 non-null   float64
 12  SALTO-05          3794 non-null   float64
 13  SALTO-06          3794 non-null   float64
 14  SALTO-07          3794 non-null   float64
 15  SALTO-08          3794 non-null   float64
 16  SALTO-09        

#### Vector de Target

In [28]:
# Quedo con todas las columnas de df menos las de X (da 'SALTO')
y = df.drop(columns=X.columns)

In [29]:
y.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3794 entries, 2011-02-10 to 2021-06-30
Freq: D
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SALTO   3794 non-null   float64
dtypes: float64(1)
memory usage: 59.3 KB


#### Lista de métricas

In [30]:
# Lista para las métricas
metricas = []

#### Dataframe de pronósticos (y_preds)

In [31]:
y_preds_df = pd.DataFrame()

### Parámetros para los lazos de 'fh' y de nro. de 'lags'

#### Conjunto de variables (lags) a utilizar

In [32]:
# Conjunto con los números de lags a utilizar
lags_s = [40, 30, 20, 10]

#### Conjunto de horizontes de pronóstico

In [33]:
# Horizontes de predicción a explorar
# fh_s = [56, 28, 14, 7]
fh_s = [56, 7]

### Lazo principal

In [34]:
# Lazo de horizontes de predicción
for fh in fh_s:
    
    # Impresión de control
    print('\nProcesando horizonte de ', fh, ' días')
    
    # Lazo de lags
    for lags in lags_s:
        
        #### Lista de modelos
        modelos = []
        
        # Impresión de control
        print('\nProcesando lags de ', lags, ' variables')
        
        # Selecciono las columnas a usar de la matriz de predictoras 'X'
        features = [col for col in X.columns if (col[0:6]=='SALTO-' and int(col[6:8]) <= lags)] # Agrego variables del dataset
 
        # Me quedo con las predictoras correspondientes al lags en proceso 
        X_features = X[features + features_generadas]
        
#         print('\nMatriz X_features:')
#         X_features.info()
        
        ### Puntos de testing (x fh) a reservar
        # Nro de puntos de prueba
        t = 1

        # Cantidad de registros a reservar
        nTesting = t * fh

        ### Conjuntos de Training y de Testing

        #### Training
        X_train = X_features.iloc[0:-nTesting,]
        y_train = y.iloc[0:-nTesting,]

#         # Imprimo info de los conjuntos de training
#         print('\nMatriz X_train:')
#         X_train.info()
#         print('\nVector y_train:')
#         y_train.info()

        #### Testing
        X_test = X_features.iloc[-nTesting:,]
        y_test = y.iloc[-nTesting:,]

#         # Imprimo info de los conjuntos de test
#         print('\nMatriz X_test:')
#         X_test.info()
#         print('\nVector y_test:')
#         y_test.info()
        
        ### Entrenamiento / ajuste de los modelos
        print('\nAjuste de modelos...')

        # Ciclo la lista de regresores
        for regre in regresores:

            # Impresión de Control
            print('\nRegresor: ', type(regre).__name__)

            # Lista para los modelos diarios (submodelos)
            submodelos = []

            for dia in range(0, fh):

                # Reseteo el regresor seleccionado
                regresor = clone(regre)

                # Impresión de Control
                print('Ajustando día', dia)

                # Conjuntos de entrenamiento
                y_train_submodelo = y_train.shift(periods=-dia).iloc[0:(-1-dia)]
                X_train_submodelo = X_train.iloc[0:(-1-dia)]

        #         print(X_train_submodelo.info(), y_train_submodelo.info())

                # Ajusto un modelo y lo agrego a la lista de submodelos
                submodelos.append(regresor.fit(X_train_submodelo, y_train_submodelo.values.ravel()))

                # *** Fin lazo de los días

            # Agrego la lista de submodelos a la lista de modelos
            modelos.append(
                {'regresor': type(regresor).__name__
                 ,'lags':lags
                 ,'fh': fh
                 ,'submodelos': submodelos}
                )

            # *** Fin lazo regresores  

        # Impresión de control
        print('\nFin ajuste de modelos.')   
         
        ### Pronóstico / predicciones

        #### Calculo x_pred, como la primera fila de X_test
        x_pred = X_test.iloc[0:1]

        # Guardo y_test en el dataframe de predicciones
        y_preds_df[str(fh) + '_' + str(lags) + '_' + 'y_test'] = y_test

        # Impresión de control
        print('\nPredicción de valores...')

        # Ciclo los modelos ajustados
        for modelo in modelos:

            # Impresión de Control
            print('\nPrediciendo Regresor: ', modelo['regresor'])

            # Lista para las predicciones diarias de cada submodelo
            y_pred = []

            for dia in range(0, fh):

                # Impresión de Control
                print('Prediciendo día:', dia, ', Valor:', modelo['submodelos'][dia].predict(x_pred)[0])

                # Predigo y agrego a la lista de y_pred
                y_pred.append(modelo['submodelos'][dia].predict(x_pred)[0])

                # *** Fin lazo de los días

            # Convierto la lista de predicciones a una serie
            y_pred_series = pd.Series(y_pred, index = y_test.index)

            # Agrego la predicción al dataframe de predicciones
            y_preds_df[str(fh)+'_' + str(lags) + '_' + modelo['regresor']] = y_pred_series

            # Calculo metricas
            MAE = mae(y_test, y_pred_series) 
            MSE = mse(y_test, y_pred_series)
            RMSE = rmse(y_true=y_test, y_pred=y_pred_series) 
            MASE = mase(y_true=y_test, y_pred=y_pred_series, y_train=y_train)

            metricas.append({'fh': fh
                             ,'lags':lags
                             ,'regresor': modelo['regresor']
                             ,'MAE': MAE
                             ,'MSE': MSE
                             ,'RMSE': RMSE
                             ,'MASE': MASE
                            })

            # *** Fin lazo modelos

            # Ploteo los resultados del regresor/lags
            pd.concat([y.iloc[-(fh+5):], y_pred_series.to_frame(name='SALTO_pred')], axis=1).iplot(
                mode='lines+markers', size=5.0
                ,title='Algoritmo: ' + modelo['regresor']  + ',  features: ' + str(lags) + ',  fh:'+ str(fh) + ', MAE: ' + f'{MAE:0.4f}' + ', RMSE: ' + f'{RMSE:0.4f}'
                ,yTitle='metros'
                ,xTitle='Fecha'
                ,dimensions=(950, 350)
                )


        # Impresión de control
        print('\nFin predicción.')  
        
    
    # Impresión de control
    print('\nFin procesamiento de lags.')
    
    
    
    # *** Fin lazo horizones
    
# Impresión de control
print('\nFin procesamiento de horizontes')


Procesando horizonte de  56  días

Procesando lags de  40  variables

Ajuste de modelos...

Regresor:  KNeighborsRegressor
Ajustando día 0
Ajustando día 1
Ajustando día 2
Ajustando día 3
Ajustando día 4
Ajustando día 5
Ajustando día 6
Ajustando día 7
Ajustando día 8
Ajustando día 9
Ajustando día 10
Ajustando día 11
Ajustando día 12
Ajustando día 13
Ajustando día 14
Ajustando día 15
Ajustando día 16
Ajustando día 17
Ajustando día 18
Ajustando día 19
Ajustando día 20
Ajustando día 21
Ajustando día 22
Ajustando día 23
Ajustando día 24
Ajustando día 25
Ajustando día 26
Ajustando día 27
Ajustando día 28
Ajustando día 29
Ajustando día 30
Ajustando día 31
Ajustando día 32
Ajustando día 33
Ajustando día 34
Ajustando día 35
Ajustando día 36
Ajustando día 37
Ajustando día 38
Ajustando día 39
Ajustando día 40
Ajustando día 41
Ajustando día 42
Ajustando día 43
Ajustando día 44
Ajustando día 45
Ajustando día 46
Ajustando día 47
Ajustando día 48
Ajustando día 49
Ajustando día 50
Ajustando día 51
Aj

Ajustando día 21
Ajustando día 22
Ajustando día 23
Ajustando día 24
Ajustando día 25
Ajustando día 26
Ajustando día 27
Ajustando día 28
Ajustando día 29
Ajustando día 30
Ajustando día 31
Ajustando día 32
Ajustando día 33
Ajustando día 34
Ajustando día 35
Ajustando día 36
Ajustando día 37
Ajustando día 38
Ajustando día 39
Ajustando día 40
Ajustando día 41
Ajustando día 42
Ajustando día 43
Ajustando día 44
Ajustando día 45
Ajustando día 46
Ajustando día 47
Ajustando día 48
Ajustando día 49
Ajustando día 50
Ajustando día 51
Ajustando día 52
Ajustando día 53
Ajustando día 54
Ajustando día 55

Fin ajuste de modelos.

Predicción de valores...

Prediciendo Regresor:  KNeighborsRegressor
Prediciendo día: 0 , Valor: 23.764000000000003
Prediciendo día: 1 , Valor: 23.764000000000003
Prediciendo día: 2 , Valor: 23.764000000000003
Prediciendo día: 3 , Valor: 23.764000000000003
Prediciendo día: 4 , Valor: 23.764000000000003
Prediciendo día: 5 , Valor: 23.764000000000003
Prediciendo día: 6 , Valor: 2


Prediciendo Regresor:  Lasso
Prediciendo día: 0 , Valor: 23.136153484538998
Prediciendo día: 1 , Valor: 23.107865208548933
Prediciendo día: 2 , Valor: 23.08124444197882
Prediciendo día: 3 , Valor: 23.058397545668026
Prediciendo día: 4 , Valor: 23.05799639091504
Prediciendo día: 5 , Valor: 23.057539937791926
Prediciendo día: 6 , Valor: 23.05691467268968
Prediciendo día: 7 , Valor: 23.056260341303428
Prediciendo día: 8 , Valor: 23.05558764439043
Prediciendo día: 9 , Valor: 23.055010171230574
Prediciendo día: 10 , Valor: 23.054555398695133
Prediciendo día: 11 , Valor: 23.054180431637217
Prediciendo día: 12 , Valor: 23.053820437025625
Prediciendo día: 13 , Valor: 23.053459171131717
Prediciendo día: 14 , Valor: 23.05309663146584
Prediciendo día: 15 , Valor: 23.052760051051827
Prediciendo día: 16 , Valor: 23.052384101471986
Prediciendo día: 17 , Valor: 23.052012297389066
Prediciendo día: 18 , Valor: 23.05168837470357
Prediciendo día: 19 , Valor: 23.051368764625476
Prediciendo día: 20 , Valo


Prediciendo Regresor:  LGBMRegressor
Prediciendo día: 0 , Valor: 23.74974393282804
Prediciendo día: 1 , Valor: 23.74501274528263
Prediciendo día: 2 , Valor: 23.635605769417882
Prediciendo día: 3 , Valor: 23.593868762566444
Prediciendo día: 4 , Valor: 22.623190515162747
Prediciendo día: 5 , Valor: 23.59409808044655
Prediciendo día: 6 , Valor: 22.658330600967144
Prediciendo día: 7 , Valor: 23.7832811519161
Prediciendo día: 8 , Valor: 25.635574605122343
Prediciendo día: 9 , Valor: 23.53919839388648
Prediciendo día: 10 , Valor: 19.63482882452674
Prediciendo día: 11 , Valor: 23.607552543724264
Prediciendo día: 12 , Valor: 23.608951790057368
Prediciendo día: 13 , Valor: 23.830757012871846
Prediciendo día: 14 , Valor: 25.10660211045154
Prediciendo día: 15 , Valor: 23.32496630668444
Prediciendo día: 16 , Valor: 23.75220436996548
Prediciendo día: 17 , Valor: 23.39250352577714
Prediciendo día: 18 , Valor: 22.69558347480925
Prediciendo día: 19 , Valor: 22.294272409396875
Prediciendo día: 20 , Va


Prediciendo Regresor:  LinearRegression
Prediciendo día: 0 , Valor: 23.756389958783984
Prediciendo día: 1 , Valor: 23.736837066709995
Prediciendo día: 2 , Valor: 23.734522817656398
Prediciendo día: 3 , Valor: 23.72429240308702
Prediciendo día: 4 , Valor: 23.671690925955772
Prediciendo día: 5 , Valor: 23.63372477144003
Prediciendo día: 6 , Valor: 23.601489879190922
Prediciendo día: 7 , Valor: 23.581430435180664
Prediciendo día: 8 , Valor: 23.59047767519951
Prediciendo día: 9 , Valor: 23.606475919485092
Prediciendo día: 10 , Valor: 23.59510473906994
Prediciendo día: 11 , Valor: 23.55227141082287
Prediciendo día: 12 , Valor: 23.51380990445614
Prediciendo día: 13 , Valor: 23.48144592344761
Prediciendo día: 14 , Valor: 23.489499807357788
Prediciendo día: 15 , Valor: 23.524052783846855
Prediciendo día: 16 , Valor: 23.56017069518566
Prediciendo día: 17 , Valor: 23.548241421580315
Prediciendo día: 18 , Valor: 23.506653122603893
Prediciendo día: 19 , Valor: 23.475707709789276
Prediciendo día: 

...


Prediciendo Regresor:  RandomForestRegressor
Prediciendo día: 0 , Valor: 23.78639999999999
Prediciendo día: 1 , Valor: 23.68079999999998
Prediciendo día: 2 , Valor: 23.706599999999998
Prediciendo día: 3 , Valor: 23.592799999999997
Prediciendo día: 4 , Valor: 23.611600000000003
Prediciendo día: 5 , Valor: 23.71659999999999
Prediciendo día: 6 , Valor: 23.737700000000007



Prediciendo Regresor:  Ridge
Prediciendo día: 0 , Valor: 23.82837664434888
Prediciendo día: 1 , Valor: 23.794856598450266
Prediciendo día: 2 , Valor: 23.77601363644436
Prediciendo día: 3 , Valor: 23.761976486373342
Prediciendo día: 4 , Valor: 23.74256246259027
Prediciendo día: 5 , Valor: 23.71509599811438
Prediciendo día: 6 , Valor: 23.683287196009218



Prediciendo Regresor:  SVR
Prediciendo día: 0 , Valor: 23.63071030071157
Prediciendo día: 1 , Valor: 23.382099849330633
Prediciendo día: 2 , Valor: 23.334082030382135
Prediciendo día: 3 , Valor: 23.28607244370298
Prediciendo día: 4 , Valor: 23.220675791054184
Prediciendo día: 5 , Valor: 23.160864828875486
Prediciendo día: 6 , Valor: 23.115091169609236



Prediciendo Regresor:  XGBRegressor
Prediciendo día: 0 , Valor: 23.782732
Prediciendo día: 1 , Valor: 23.805628
Prediciendo día: 2 , Valor: 23.80966
Prediciendo día: 3 , Valor: 23.777205
Prediciendo día: 4 , Valor: 23.735851
Prediciendo día: 5 , Valor: 23.753277
Prediciendo día: 6 , Valor: 23.713911



Fin predicción.

Fin procesamiento de lags.

Fin procesamiento de horizontes


In [35]:
# Guardo el dataframe de predicciones en disco
y_preds_df.to_csv(path_grabacion + 'y_preds_df.csv')

In [ ]:
y_preds_df.iplot()

### Análisis de las métricas

In [ ]:
# Creo un dataframe con la lista de métricas
metricas_df = pd.DataFrame(metricas)

In [ ]:
# Guardo el dataframe de metricas en disco
metricas_df.to_csv(path_grabacion + 'metricas_df.csv', index=False)

In [ ]:
metricas_df.sort_values('MAE').style.set_properties(**{'text-align': 'left'}).hide_index()

#### Mínimos MAE

In [ ]:
# Listo las combinaciones regresor/lags para el horizonte de pronóstico de 48 días 
metricas_df.loc[metricas_df['fh']==7].sort_values(['fh', 'MAE'])

#### Desempeño de cada regresor

In [ ]:
for lags in lags_s:
    metricas_df.loc[metricas_df['lags']==lags].sort_values(by=['fh', 'MAE']).iplot(kind='bar'
                                         ,x=['fh', 'regresor']
                                         ,y=['MAE', 'RMSE']
                                         ,title='Desempeño de cada regresor, por horizonte de predicción y nro. de lags a '+ str(fh) + ' días y con ' + str(lags) + ' variables' 
                                         ,yTitle='metros', xTitle='Regresor'
                                         ,dimensions=(950, 500)
                                         ,theme='pearl'
                                        )

In [ ]:
# Pivoteo el dataframe de métricas, para agrupar los MAE por nro. de lags
metricas_df.pivot_table(index=['fh', 'lags'], columns='regresor', values='MAE')

#### Máximos, promedio y mínimos MAE por cada horizonte y número de lags

In [ ]:
# Ploteo el mínimo MAE por cada nro. de lags
metricas_df.groupby(['fh', 'lags'])['MAE'].agg(['max', 'mean', 'min']).iplot(kind='bar'
                                                 ,title='MAE (máximo, promedio y mínimo), por horizonte de pronóstico y por número de features (lags)' 
                                                 ,yTitle='metros'
                                                 , xTitle='(horizonte de pronóstico, nro. de lags)'
                                                 ,dimensions=(950, 350)
                                                 ,theme='pearl'
                                                )

In [ ]:
# Ploteo los MAE por nro de lags y por regresor
metricas_df.iplot(kind='bar'
                  ,x=['fh', 'lags', 'regresor']
                  ,y=['MAE']
                  ,title='MAE de cada regresor, por horizonte de pronóstico y por número de features (lags)' 
                  ,yTitle='metros'
                  ,xTitle='(horizonte de pronóstico, nro. de lags, regresor)'
                  ,dimensions=(950, 700)
                  ,theme='pearl'
                  ,barmode='group'
                  )

In [ ]:
# Pivoteo el dataframe de métricas, para agrupar los MAE por nro. de lags
metricas_df.pivot_table(index=['fh', 'lags'], columns='regresor', values='MAE')

#### MAE de cada regresor por horizonte y por conjunto de lags

In [ ]:
# Ploteo los MAE para cada regresor, para cada nro. de lags
metricas_df.pivot_table(index=['fh', 'lags'], columns='regresor', values='MAE').iplot(kind='bar'
                                                                              ,title='MAE de cada regresor, por número de features (lags)'
                                                                              ,yTitle='metros'
                                                                              ,xTitle='(horizonde de pronóstico, nro. de lags)'
                                                                              ,dimensions=(950, 350)
                                                                              ,theme='pearl'
                                                                              )

In [ ]:
# Busco el mínimo MAE
metricas_df[metricas_df['MAE']==metricas_df['MAE'].min()]

In [ ]:
# Busco el mínimo MAE
for fh in fh_s:
    metricas_df_fh = metricas_df[metricas_df['fh']==fh]
    print(metricas_df_fh[metricas_df_fh['MAE']==metricas_df_fh['MAE'].min()])

## Fin

In [ ]:
print('\n¡FIN!', '(al fin.....)')